# selenium을 이용한 웹크롤링
 - 다음페이지 버튼 클릭하여 모든 페이지 크롤링하기

In [1]:
import time

from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import requests
#from flask import request

In [ ]:
#### url='https://kr.indeed.com/jobs?q=python&limit=50'

options=ChromeOptions()
#options.add_argument('headless')  # headless: 가상 웹브라우저 화면에 띄우지 않고 실행
options.add_argument('window-size=1920x1080')
driver=Chrome(options=options)

# driver=Chrome()
driver.get(url)  # 이동

In [2]:
url='https://kr.indeed.com/jobs?q=python&limit=50'

options=ChromeOptions()
#options.add_argument('headless')  # headless: 가상 웹브라우저 화면에 띄우지 않고 실행
options.add_argument('window-size=1920x1080')
driver=Chrome(options=options)

# driver=Chrome()
driver.get(url)  # 이동

title_list=[]
company_list=[]
location_list=[]

while True:
    wc=WebDriverWait(driver, 10) # driver가 어떤 태그를 찾을 때 최대 10초까지 기다림
    
    
    x_btn=driver.find_elements(By.XPATH, "//*[@id='popover-x']/button")
    print(x_btn)
    
    if len(x_btn)>0 :
        x_btn[0].click()
    
    url=driver.current_url
    
    res=requests.get(url)
    res_text=res.text
    soup=BeautifulSoup(res_text, "html.parser")
    
#         time.sleep(0.5)
    
    jobs_tag=soup.find_all("div",{"class":"jobsearch-SerpJobCard"})
    # print(jobs_tag)
    
    for job in jobs_tag:
        title=job.find("h2", {"class":"title"}).find("a")["title"]
        info=job.find("div",{"class":"sjcl"})
        company = info.find("span",{"class":"company"}).get_text('\n', strip=True) if len(info.find("span",{"class":"company"})) > 0 else ""
#         location = info.find("span",{"class":"location accessible-contrast-color-location"}).get_text('\n', strip=True) if len(info.find("span",{"class":"location accessible-contrast-color-location"})) > 0 else "" 
        location = info.find("div",{"class":"recJobLoc"})["data-rc-loc"]
        title_list.append(title)
        company_list.append(company)
        location_list.append(location)
#         print("title : "+title, "/ company : "+company, "/ location : "+location)

    
    # 다음페이지 버튼을 찾고 클릭하는 과정
    li=driver.find_elements(By.XPATH, '//*[@id="resultsCol"]/nav/div/ul/li')
    next_btn=li[-1].find_elements(By.CLASS_NAME, 'np')
    
    
    if len(next_btn) > 0:
        next_btn[0].click()   # 클릭 동작
        time.sleep(0.5)  # 버튼 클릭 후 0.5 기다림
    else:
        break

[]
[<selenium.webdriver.remote.webelement.WebElement (session="17020fd028a35bbe9f684db4fa817e52", element="e3fe30ac-3186-494e-bb12-31a983f1c9fc")>]
[]
[]
[]
[]
[]
[]
[]
[]


In [3]:
import pandas as pd

list={"title":title_list, "company":company_list, "location":location_list}
df=pd.DataFrame(list)
df.to_excel(excel_writer="IDNEED_SCRAP.xlsx") 
# df.to_csv('jobs.csv', sep='/')  # csv: text로 저장

In [4]:
df

,title,company,location
0,2021년도 상반기 한전KDN 신입사원,한전KDN,대한민국
1,데이터 분석가/사이언티스트(신입/인턴 가능),쏘카,대한민국
2,"빅데이터, 인공지능(AI) 외 신입/경력 인턴",스칼라웍스,대한민국
3,"[라라와 함께] (주)라라 인공지능(AI), 인공지능(AI) 개발자",라라,부산 수영구
4,데이터 분석가,스포카,부산 해운대구
...,...,...,...
496,AppArmor Security Engineer,Canonical - Jobs,서울
497,[카카오 클라우드]DBaaS(DataBase As A Service) 개발자 모집,카카오,성남 판교역
498,리눅스 시스템 운영관리 과장~차장급,씨디네트웍스,서울 강남구
499,HW/SW 분야 직무 교육 전문강사,에이치에스글로벌,서울 강남구
